In [1]:
import pandas as pd 
import xlrd 
import os 
import re 

In [10]:
def turningTable(age,meanCols,eachFile, df): 
    parsedDF = df.iloc[1:, meanCols]
    parsedDF.columns = ['Nutrition'] + age
    tempTB = parsedDF.T
    tempTB.reset_index(inplace = True)
    tempTB.columns = tempTB.iloc[0].apply(lambda x: re.sub("[\u3000\t\n]+", "", x).rstrip())
    tempTB.drop(tempTB.index[0], inplace = True)
    tempTB.rename(columns = {'Nutrition':'age'}, inplace = True)
    tempTB['year'] = re.findall('_(\d+)',eachFile)[0]
    
    if re.findall('total', eachFile):
        tempTB['gender'] = 'All'
    elif re.findall('female', eachFile): 
        tempTB['gender'] = 'Female'
    else: 
        tempTB['gender'] = 'Male'
    return tempTB

In [11]:
df_2017 = {fileName for fileName in os.listdir() if re.findall('_2017.xlsx',fileName)}

In [12]:
df_2017

{'jp_FoodG_female_2017.xlsx',
 'jp_FoodG_male_2017.xlsx',
 'jp_FoodG_total_2017.xlsx'}

In [13]:
jp_2017 = list()

for eachFile in df_2017: 
    
    df = pd.ExcelFile(eachFile).parse(0)
    age = list(df.iloc[4,3:].dropna())
    df.dropna(thresh = 20, inplace = True)
    meanCols =[0]+[i for i in range(5,df.shape[1], 3)]
        
    jp_2017.append(turningTable(age,meanCols,eachFile, df))

In [14]:
jp_2017DF = pd.concat(jp_2017)

In [16]:
jp_2017DF.columns

Index(['age', '総量', '動物性食品', '植物性食品', '穀類', '米・加工品', '米', '米加工品', '小麦・加工品',
       '小麦粉類',
       ...
       '調味料', 'ソース', 'しょうゆ', '塩', 'マヨネ－ズ', '味噌', 'その他の調味料', '香辛料・その他', 'year',
       'gender'],
      dtype='object', name=0, length=134)

In [17]:
df_else = tuple(fileName for fileName in os.listdir() if re.findall('201[3-6]',fileName))

In [18]:
jp_else = list()

for eachFile in df_else: 
    df = pd.ExcelFile(eachFile).parse(0)
    age = list(df.iloc[4,5:].dropna())
    df.dropna(thresh = 20, inplace = True)
    meanCols =[0]+[i for i in range(5,df.shape[1], 3)]
        
    jp_else.append(turningTable(age,meanCols,eachFile, df))

In [19]:
jp_elseDF = pd.concat(jp_else)

In [20]:
df_2011 = tuple(fileName for fileName in os.listdir() if re.findall('2011',fileName))

In [21]:
jp_2011 = list()

for eachFile in df_2011: 
    df = pd.ExcelFile(eachFile).parse(0)
    age = list(df.iloc[4,4:].dropna())
    df.dropna(thresh = 20, inplace = True)
    meanCols =[0]+[i for i in range(4,df.shape[1], 2)]
    
    jp_2011.append(turningTable(age,meanCols,eachFile, df))

In [22]:
jp_2011DF = pd.concat(jp_2011)

In [23]:
df_2012 = tuple(fileName for fileName in os.listdir() if re.findall('2012',fileName))
jp_2012 = list()

for eachFile in df_2012: 
    
    df = pd.ExcelFile(eachFile).parse(0)
    
    age = list(df.iloc[5,5:].dropna())
    df.dropna(thresh = 20, inplace = True)
    meanCols =[0]+[i for i in range(5,df.shape[1], 3)]
        
    jp_2012.append(turningTable(age,meanCols,eachFile, df))

In [24]:
jp_2012DF = pd.concat(jp_2012)

In [51]:
for i,x in enumerate(jp_2011DF.columns): 
    if x not in jp_2012DF.columns: 
        print(i)

132


In [55]:
jp_2011DF.drop(jp_2011DF.columns[132], axis = 1, inplace = True)

In [56]:
a = jp_2012DF.append(jp_elseDF)
b = a.append(jp_2011DF)
finalTB = b.append(jp_2017DF)

In [57]:
finalTB.to_csv('JP_FoodG.csv', index = False)